In [2]:
from visual_transformer import *

In [13]:
class VisionCanvases(nn.Module):
    def __init__(self, num_canvases=3, num_channels=3, width=224, height=224, device='cpu'):
        super(VisionCanvases, self).__init__()

        self.current_index = 0
        self.empty = True
        
        self.num_canvases=num_canvases
        self.num_channels=num_channels
        self.width=width
        self.height=height
        
        canvases = torch.zeros(1, device=device) # placeholder
        self.register_buffer('canvases', canvases)

    def get_device(self):
        return self.canvases.device

    def is_empty(self):
        return self.empty

    def store(self, img_batch):
        if self.is_empty():
            nc = img_batch.unsqueeze(0).repeat(self.num_canvases, 1, 1, 1, 1)
            self.register_buffer('canvases', nc)
        else:
            self.current_index = (self.current_index + 1) % self.num_canvases
            self.canvases[self.current_index] = img_batch
        self.empty = False

    def __getitem__(self, ind):
        if self.is_empty():
            raise ValueError("can't return canvases from an empty container")
        if (ind >= self.num_canvases) or (ind <= -1 - self.num_canvases):
            raise ValueError("index out of bounds")

        newind = (self.current_index - ind) % self.num_canvases
        return self.canvases[newind]

    # should not be used; use store instead
    def forward(self, img_batch):
        self.store(img_batch)
        return self[0] # the stored value


In [14]:
vc = VisionCanvases(3)

In [35]:
# ok, problem is solved.

In [1]:
# new problem: the damn thing only works with forward pass; on backward pass, it complains about in-place modification or whatever
# new option below

In [4]:
a = torch.randn(2, 3, 4)
a.cuda()
a.device

device(type='cpu')

In [5]:
class TensorWrapper:
    def __init__(self, L):
        self.L = L

    def get_device(self):
        return self.L[0].device

    def to(self, device):
        for i in range(len(self.L)):
            self.L[i] = self.L[i].to(device)

    def cpu(self):
        return self.to('cpu')

    def cuda(self):
        return self.to('cuda')
        

In [8]:
class VisionCanvases(nn.Module):
    def __init__(self, num_canvases=3, num_channels=3, width=224, height=224, device='cpu'):
        super(VisionCanvases, self).__init__()

        self.current_index = 0
        self.empty = True
        
        self.num_canvases=num_canvases
        self.num_channels=num_channels
        self.width=width
        self.height=height
        
        tw = TensorWrapper([torch.zeros(1, num_channels, width, height, device=device) for i in range(num_canvases)])
        self.register_buffer('tw', tw)

    def get_device(self):
        return self.tw.L[0].device

    def is_empty(self):
        return self.empty

    def store(self, img_batch):
        if self.is_empty():
            for i in range(self.num_canvases):
                self.tw.L[i] += img_batch
        else:
            self.tw.L.append(img_batch)
            self.tw.L = self.tw.L[1:]
        self.empty = False

    def __getitem__(self, ind):
        if self.is_empty():
            raise ValueError("can't return canvases from an empty container")
        if (ind >= self.num_canvases) or (ind <= -1 - self.num_canvases):
            raise ValueError("index out of bounds")

        newind = (self.num_canvases - 1 - ind) % self.num_canvases
        return self.tw.L[newind]

    # should not be used; use store instead
    def forward(self, img_batch):
        self.store(img_batch)
        return self[0] # the stored value


In [9]:
vc = VisionCanvases(3)

TypeError: cannot assign '__main__.TensorWrapper' object to buffer 'tw' (torch Tensor or None required)

In [10]:
 # shucks